In [80]:
%load_ext autoreload
%autoreload 2
import os
import sys

import os
import numpy as np
import pandas as pd
# Custom utils
from utils.simulator.simulator import MCSimulation
# Tf imports
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow import keras

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [81]:
def custom_derivative_model(x, model, multioutput = True, mode = 'centered'):
    h = 1e-4
    # Size x
    x_dim, y_dim = x.shape
    # Gradient vector
    if multioutput:
        gradient = np.zeros((x_dim, y_dim, y_dim))
    else:
        gradient = np.zeros((x_dim, y_dim, 1))
    for i in range(y_dim):
        # Vector for partial derivative estimation
        offset_tensor = np.zeros((x_dim, y_dim))
        offset_tensor[:, i] = h
        offset_tensor = tf.convert_to_tensor(offset_tensor,
                                             dtype = tf.float32)
        # Constantes:
        denominator = h
        if mode == 'progressive':
            numerator = model(
                tf.math.add(x, offset_tensor)
            ) - model(
                x
            ) 
        elif mode == 'regressive':
            numerator = model(
                x
            ) - model(
                tf.math.subtract(x, offset_tensor)
            )
        elif mode == 'centered':
            numerator = tf.math.subtract(
                model(
                    tf.math.add(x, offset_tensor)
                ), model(
                    tf.math.subtract(x, offset_tensor)
                )
            )
        denominator = 2 * h
        gradient [:, i, :] = numerator / denominator
    gradient = tf.convert_to_tensor(gradient,
                                        dtype = tf.float32)
    return gradient

In [82]:
input_layer = keras.Input(shape = (2,), name='input_nn')
intermediate_layer = layers.Dense(
    units = 64, 
    activation = 'tanh', 
    name = 'first_dense'
)(input_layer)
output_layer = layers.Dense(
    units = 1,
    activation = 'tanh',
    name = 'second_dense'
)(intermediate_layer)
custom_model = keras.Model(
    inputs=[input_layer],
    outputs=[output_layer],
    name = 'test_model'
)

In [83]:
# Domain -5:5
x = tf.convert_to_tensor(
    np.array(range(0,20)).reshape((10,2)),
    dtype = tf.float32    
)
# Test 
xs = tf.Variable(x, trainable = True, name = 'x')
with tf.GradientTape() as tape, tf.GradientTape() as tape_2:
    tape.watch(xs)
    tape_2.watch(xs)
    y = custom_model(xs)
# This represents dV/dX
grads = tape.gradient(y, {
    'x':xs
})

In [84]:
custom_grads = custom_derivative_model(x, custom_model, multioutput=False)

In [85]:
tf.reshape(custom_grads, (10, 2))

<tf.Tensor: shape=(10, 2), dtype=float32, numpy=
array([[-0.03927853,  0.01263339],
       [-0.03611669,  0.00147149],
       [-0.041686  ,  0.02179295],
       [-0.05826354,  0.04939735],
       [-0.06973743,  0.06951392],
       [-0.07539988,  0.07949769],
       [-0.07435679,  0.08180737],
       [-0.07320195,  0.08258969],
       [-0.07346272,  0.07789582],
       [-0.07092953,  0.07834285]], dtype=float32)>

In [86]:
grads

{'x': <tf.Tensor: shape=(10, 2), dtype=float32, numpy=
 array([[-0.03933419,  0.01261395],
        [-0.03524638,  0.00228759],
        [-0.04136173,  0.02109241],
        [-0.05845135,  0.0490886 ],
        [-0.06996457,  0.0689986 ],
        [-0.07469031,  0.0788607 ],
        [-0.07545307,  0.08193812],
        [-0.07445842,  0.08145425],
        [-0.07293446,  0.07949664],
        [-0.07145852,  0.07718682]], dtype=float32)>}